##Initial configuration

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

os.chdir(f'/content/drive/MyDrive/NLP/project')
os.getcwd()

'/content/drive/MyDrive/NLP/project'

In [2]:
!pip install transformers datasets sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing insta

In [3]:
# Import delle librerie
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns

# Caricamento del dataset
dataset = load_dataset("neural-bridge/rag-dataset-12000")


2025-05-13 14:13:49.307266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747145629.536649      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747145629.614865      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

(…)-00000-of-00001-9df3a936e1f63191.parquet:   0%|          | 0.00/23.1M [00:00<?, ?B/s]

(…)-00000-of-00001-af2a9f454ad1b8a3.parquet:   0%|          | 0.00/5.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2400 [00:00<?, ? examples/s]

##Preprocess

In [5]:
train_set = dataset["train"]
test_set = dataset["test"]

df_train = pd.DataFrame(train_set)
df_test = pd.DataFrame(test_set)

# Rimozione di record incompleti
df_train = df_train.dropna(subset=["question", "context", "answer"])

df_subset = df_train.sample(50, random_state=42).reset_index(drop=True)

# Create prompt for "question only"
df_subset["prompt_only_question"] = df_subset["question"].apply(
    lambda q: f"Answer the following question:\n\nQuestion:\n{q}"
)

# Create prompt for "question + context"
df_subset["prompt_question_context"] = df_subset.apply(
    lambda row: f"Answer the following question using the context.\n\nContext:\n{row['context']}\n\nQuestion:\n{row['question']}",
    axis=1
)

# Preview the first example
print(df_subset[["prompt_only_question", "prompt_question_context"]].iloc[0])

prompt_only_question       Answer the following question:\n\nQuestion:\nH...
prompt_question_context    Answer the following question using the contex...
Name: 0, dtype: object


We write the code to compare the behaviour of a LLM who exploit context given to answer and one that doesn't use the context.

In [6]:
model_name = "microsoft/Phi-3-mini-4k-instruct"  # you can change this to any causal LLM
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",        # comment this line if you don't have GPU
    torch_dtype="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create generation pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Generate responses for each example
responses_question_only = []
responses_question_context = []

for i, row in df_subset.iterrows():
    print(f"Generating for example {i+1}/50...")

    # Generate answer for "question only"
    output_q = generator(
        row["prompt_only_question"],
        max_new_tokens=300,
        temperature=0.3,
        do_sample=True
    )[0]["generated_text"]
    responses_question_only.append(output_q)

    # Generate answer for "question + context"
    output_qc = generator(
        row["prompt_question_context"],
        max_new_tokens=300,
        temperature=0.3,
        do_sample=True
    )[0]["generated_text"]
    responses_question_context.append(output_qc)

# Add results to dataframe
df_subset["response_question_only"] = responses_question_only
df_subset["response_question_context"] = responses_question_context

print("Generation complete!")

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda


Generating for example 1/50...
Generating for example 2/50...
Generating for example 3/50...
Generating for example 4/50...
Generating for example 5/50...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Generating for example 6/50...
Generating for example 7/50...
Generating for example 8/50...
Generating for example 9/50...
Generating for example 10/50...
Generating for example 11/50...
Generating for example 12/50...
Generating for example 13/50...
Generating for example 14/50...
Generating for example 15/50...
Generating for example 16/50...
Generating for example 17/50...
Generating for example 18/50...
Generating for example 19/50...
Generating for example 20/50...
Generating for example 21/50...
Generating for example 22/50...
Generating for example 23/50...
Generating for example 24/50...
Generating for example 25/50...
Generating for example 26/50...
Generating for example 27/50...
Generating for example 28/50...
Generating for example 29/50...
Generating for example 30/50...
Generating for example 31/50...
Generating for example 32/50...
Generating for example 33/50...
Generating for example 34/50...
Generating for example 35/50...
Generating for example 36/50...
Generating f

In [7]:
# Install nltk if not yet installed
!pip install nltk

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Download punkt tokenizer (first time only)
nltk.download("punkt")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
def calculate_bleu(reference_text, candidate_text):
    # Tokenize reference and candidate
    reference_tokens = nltk.word_tokenize(reference_text.lower())
    candidate_tokens = nltk.word_tokenize(candidate_text.lower())

    # BLEU with smoothing
    smoothie = SmoothingFunction().method4
    score = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothie)
    return score

# Store BLEU scores
bleu_scores_question_only = []
bleu_scores_question_context = []

for i, row in df_subset.iterrows():
    reference = row["answer"]

    # Calculate BLEU for question only
    bleu_q = calculate_bleu(reference, row["response_question_only"])
    bleu_scores_question_only.append(bleu_q)

    # Calculate BLEU for question + context
    bleu_qc = calculate_bleu(reference, row["response_question_context"])
    bleu_scores_question_context.append(bleu_qc)

# Add BLEU scores to dataframe
df_subset["bleu_question_only"] = bleu_scores_question_only
df_subset["bleu_question_context"] = bleu_scores_question_context

print("BLEU scoring complete!")


BLEU scoring complete!


In [9]:
print(f"Average BLEU score (question only): {df_subset['bleu_question_only'].mean():.4f}")
print(f"Average BLEU score (question + context): {df_subset['bleu_question_context'].mean():.4f}")

Average BLEU score (question only): 0.0418
Average BLEU score (question + context): 0.0418


If the question is general the LLM could perform better because the retrieved document could be misleading or irrelevant. 
Instead the LLM with context perform better when the question needs the context to answer, sometimes question are directly connected to the context.

In [14]:
def extract_answer(text):
    # Simple heuristic: take everything after 'Answer:'
    if "Answer:" in text:
        return text.split("Answer:")[-1].strip()
    return text.strip()

# Manual review with cleaner answers
for i in range(N):
    print(f"\n Example {i+1}")
    print(f" Question:\n{df_subset['question'].iloc[i]}\n")
    print(f"Context (shortened):\n{df_subset['context'].iloc[i][:500]}...\n")
    print(f"Ground Truth Answer:\n{df_subset['answer'].iloc[i]}\n")
    print(f"Model Answer (Question Only):\n{extract_answer(df_subset['response_question_only'].iloc[i])}\n")
    print(f"Model Answer (Question + Context):\n{extract_answer(df_subset['response_question_context'].iloc[i])}\n")
    print("="*100)




 Example 1
 Question:
How long has Move My Car been transporting cars from Devonport to Devonport?

Context (shortened):
Cheap Car Transport from Mandurah to Griffith.
If you’re needing to move a car from Devonport to Devonport we are able to give you an immediate quote for this trip, as it is one of our most frequently run and major vehicle transport trips. If you’re currently looking to move to somewhere different around Australia, then we can assist with that, but we may need to calculate those prices if our database does not instantly give them to you.
When we Car Transport Interstate from Devonport to Devonpo...

Ground Truth Answer:
Move My Car has been moving cars from Devonport to Devonport for over 25 years.

Model Answer (Question Only):
The average

Model Answer (Question + Context):
Move My Car has been transporting cars from Devonport to Devonport for over 25 years.

Context:
Cheap Car Transport from Mandurah to Griffith.
If you’re needing to move a car from Devonport to 

In [15]:
!pip install numpy scipy librosa unidecode inflect
!apt-get update
!apt-get install -y libsndfile1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 9.1 MB/s eta 0:00:00


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]                
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease         
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease                        
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease                  
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease   
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease                        
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]       
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,901 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]          
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                             

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 89 not upgraded.


In [27]:
# Install Coqui TTS (opzionale ma consigliato)
!pip install TTS

from TTS.api import TTS

# Initialize a TTS model
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False, gpu=True)

# Example: take LLM response
example_text = df_subset["response_question_context"].iloc[0]

# Generate speech
tts.tts_to_file(text=example_text, file_path="tts_output.wav")

# Play audio
from IPython.display import Audio
Audio("tts_output.wav")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C
ERROR: Operation cancelled by user
 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024


/usr/local/lib/python3.11/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio

file_path = "tts_output.wav"
y, sr = librosa.load(file_path, sr=None)
print(f"Audio duration: {len(y) / sr:.2f} seconds")

# Plot only part of the signal (first N seconds)
seconds_to_plot = 5  # Change this to 10, 20 etc.
samples_to_plot = int(seconds_to_plot * sr)

# Create time array
time = np.linspace(0, seconds_to_plot, samples_to_plot)

# Plot
plt.figure(figsize=(14, 5))
plt.plot(time, y[:samples_to_plot])
plt.title(f"Waveform of first {seconds_to_plot} seconds of audio")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.show()
